In [ ]:
'''
TrainingBERT on RE:
1) extract all FrontPages from 100 S-3 filings
2) annotate gold examples
3) split 80/20
4) train
PreTraining BERT:
1) collect 1000 filings (take S-3, 8-k with relevant items), strip TitlePage
2) preTrain BERT with Match the Blanks
'''


In [1]:
from pathlib import Path
import logging

root_path = Path(r"C:\Users\Olivi\Desktop\test_set")

In [ ]:
# testing annotation process on currently available test set s-3

# parse files with s-3 parser and extract Summary and CoverPage from the the filings
# save to unannoted_s3_cover_page.txt and unannoted_s3_summary.txt
from pathlib import Path
from main.parser.filing_parsers import create_htm_filing
import re
import logging
logging.disable(level=logging.DEBUG)
fps = list(Path(r"C:\Users\Olivi\Desktop\test_set\set_s3\filings").rglob("*.htm"))
failed = []
for idx, fp in enumerate(fps):
    if idx%10 == 0:
        print(f"{idx}|{len(fps)}")
    info = {
        "form_type":"S-3",
        "extension": ".htm",
        "path": fp,
        "filing_date": "-",
        "accession_number": "-",
        "cik": "-",
        "file_number": "-",
    }
    try:
        filings = create_htm_filing(**info)
    except AttributeError as e:
        failed.append((fp, e))
    else:
        if isinstance(filings, list):
            pass
        else:
            filings = [filings]
        for filing in filings:
            sections = filing.get_sections(identifier=re.compile("description"))
            if sections:
                for section in sections:
                    with open(r"C:\Users\Olivi\Desktop\test_set\unannoted_s3_descriptions.txt", "a", encoding="utf-8") as f:
                        text = section.text_only
                        f.write("\n+++::::")
                        f.write(text)

            # cover_page = filing.get_section(identifier=re.compile("cover"))
            # if cover_page != []:
            #     with open(r"C:\Users\Olivi\Desktop\test_set\unannoted_s3_cover_page.txt", "a", encoding="utf-8") as f:
            #         cover_text = cover_page.text_only
            #         f.write("\n+++::::")
            #         f.write(cover_text)
            # summary = filing.get_section(identifier=re.compile("summary"))
            # if summary != []:
            #     with open(r"C:\Users\Olivi\Desktop\test_set\unannoted_s3_summary.txt", "a", encoding="utf-8") as f:
            #         summary_text = summary.text_only
            #         f.write("\n+++::::")
            #         f.write(summary_text)
logging.disable(logging.NOTSET)

for fail in failed:
    print(fail)






In [ ]:
# use current SpacyFilingTextSearch to add SECU, SECUQUANTITY, CONTRACT entities aswell as the regular ones: PER, ORG, ect..
# load and split by "+++:::" to get individual pages
with open(root_path / "unannoted_s3_descriptions.txt", "r", encoding="utf-8") as f:
    pages = f.read().split("\n+++::::")
    pages = list(filter(lambda x: len(x)>50, pages))

# load pipeline
from main.nlp.filing_nlp import SpacyFilingTextSearch
import json
logger = logging.getLogger("filing_nlp")
logger.setLevel(logging.INFO)
save_folder = root_path / "s3_doccano_descriptions"
if not save_folder.exists():
    save_folder.mkdir()
search = SpacyFilingTextSearch()
for i, page in enumerate(pages):
    try:
        doc = search.nlp(page)
        # convert to compatible json format for doccano
        # get json first
        j = doc.to_json()
        doccano_dict = {"text": j["text"], "ents": [[i["start"], i["end"], i["label"]] for i in j["ents"]]}
        save_path = save_folder / f"{i}.json"
        with open(save_path, "w", encoding="utf-8") as g:
            json.dump(doccano_dict, g)
    except Exception as e:
        raise e
    


 

In [3]:
from spacy.tokens import DocBin
from pathlib import Path
from sdp_utils import *
from main.nlp.filing_nlp import SpacyFilingTextSearch
import logging
logger = logging.getLogger("filing_nlp")
logger.setLevel(logging.INFO)
search = SpacyFilingTextSearch()

if not Path(r"C:\Users\Olivi\Desktop\test_set\s3_descriptions.spacy").exists():
    with open(r"C:\Users\Olivi\Desktop\test_set\unannoted_s3_descriptions.txt", "r", encoding="utf-8") as f:
        pages = f.read().split("\n+++::::")
        pages = list(filter(lambda x: len(x)>50, pages))

    docs = []
    for idx, page in enumerate(pages[:10]):
        if idx%10:
            print(f"{idx}/{len(pages)}")
        doc = search.nlp(page)
        docs.append(doc)
    # import pickle
    # from spacy.tokens import Span, Token
    # for idx, page in enumerate(pages[:2]):
    #     if idx%10:
    #         print(f"{idx}/{len(pages)}")
    #     doc = search.nlp(page)
#         with open(r"C:\Users\Olivi\Desktop\test_set\desc_test_pickle.txt", "wb") as f:
#             doc._.secu_objects = None
#             doc._.secuquantity_spans = None
#             doc._.secu_objects_map = None
#             doc._.quantity_relation_map = None
#             doc._.secuquantity_secu_map= None
#             doc._.source_quantity_relation_map= None
#             keys_to_remove = []
#             for key in doc.user_data:
#                 if len(key) >= 2:
#                     if key[1] == 'source_span_unmerged':
#                         keys_to_remove.append(key)
#             for key in keys_to_remove:
#                 doc.user_data.pop(key)
#             pickle.dump(doc, f)
#         docs.append(doc)
#     docbin = DocBin(docs=docs, store_user_data=True)
#     docbin.to_disk(r"C:\Users\Olivi\Desktop\test_set\s3_descriptions.spacy")
# else:
#     docbin = DocBin().from_disk(path=r"C:\Users\Olivi\Desktop\test_set\s3_descriptions.spacy")
#     docs = docbin.get_docs(search.nlp.vocab)


nvu_sdps = get_sdps_noun_verbal(docs)
relations = set()
for left, right, sdp_grouped, sdp in nvu_sdps:
    info = create_sdp_noun_verbal_dict(left, right, sdp_grouped, sdp)
    if info["relation"] is not None:
        relations.add(info["relation"])
    # print(info)
    print([i["token"] for i in info["sdp"]])
    # --> sdps arent the way to go ... :#)
# print(len(relations),"/", len(nvu_sdps))
# for rel in relations:
#     print(rel)

# can i hook into msgpack and register my own encoder/decoder for AliasCache and SECUObject ?
# last i could pickle but that would mean way larger files ... 




1/176


c:\Users\Olivi\Testing\sec_scraping\main\nlp\filing_nlp_alias_setter.py:1013: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  similarity = origin_token.similarity(target_token)


2/176
3/176
4/176
5/176
6/176
7/176
8/176
9/176
[Security, with, offered, Warrants, of, consequences, States]
[States, considerations, discussed, in, Supplement]
[Agreement, of, copy, file, with, authorities, in, Canada]
[Agreement, of, copy, file, with, authorities, in, States]
[Conditions, of, satisfaction, upon, made, entitlement, satisfied, Conditions]
[Conditions, of, satisfaction, upon, made, entitlement, of, Company]
[Conditions, of, satisfaction, upon, made, entitlement, issue, Company]
[Conditions, of, satisfaction, upon, made, entitlement, issue, Company]
[Conditions, of, satisfaction, upon, made, provisions, as, to, modification, of, Agreement]
[Conditions, of, satisfaction, upon, made, provisions, as, to, modification, of, Agreement, rights, of, Receipts]
[Conditions, of, satisfaction, upon, made, provisions, as, to, modification, apply, Company]
[Conditions, of, satisfaction, upon, made, provisions, as, to, consequences, tax, U.S.]
[Conditions, of, satisfaction, upon, rele